In [23]:
import alpaca_trade_api as tradeapi
import numpy as np
from scipy import optimize as opt
import pandas as pd
from pandas_datareader.famafrench import get_available_datasets
import pandas_datareader.data as web
from sklearn.linear_model import LinearRegression
import datetime 
import workdays as wkdy

In [2]:
with open("keys.txt") as keys_file :
    GENERAL_API_KEY = keys_file.readline().strip()
    SECRET_API_KEY = keys_file.readline().strip()
ALPACA_URL_BASE  = "https://paper-api.alpaca.markets"
ALPACA_DATA_URL_BASE = "https://data.alpaca.markets/v1"

In [3]:
api  = tradeapi.REST(GENERAL_API_KEY, SECRET_API_KEY, ALPACA_URL_BASE, api_version = 'v2')
account = api.get_account()

In [4]:
#use get_available_datasets() to see all the different Fama French datasets available
ff5 = web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench')
ff5 = ff5[0][::-1] #change it from a dict to a dataframe and then change it 
                   #so it's from newest to oldest
ff5.reset_index(inplace=True)

In [5]:
ff5

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,2020-02-28,-0.78,0.06,-0.78,0.32,-0.41,0.006
1,2020-02-27,-4.22,0.69,0.14,-0.35,-0.09,0.006
2,2020-02-26,-0.52,-0.79,-1.26,-0.52,-0.12,0.006
3,2020-02-25,-3.09,-0.34,-0.72,-0.63,0.03,0.006
4,2020-02-24,-3.38,0.15,-0.04,-0.37,0.20,0.006
...,...,...,...,...,...,...,...
1230,2015-04-10,0.49,-0.15,-0.28,-0.48,0.20,0.000
1231,2015-04-09,0.41,-0.71,-0.06,-0.27,0.15,0.000
1232,2015-04-08,0.37,0.47,-0.72,-0.07,-0.28,0.000
1233,2015-04-07,-0.22,-0.20,-0.16,-0.18,-0.23,0.000


In [80]:
stocks = ['LHO','NVDA','VKTX','COG','KO']

In [81]:
days_of_data = 900
data = api.get_barset(stocks,'day',limit=days_of_data)

In [95]:
dailyreturns = pd.DataFrame()
for tkr in stocks :
    dailyreturns_col = []
    for day in range(days_of_data - 1, -1, -1) :
        dailyreturns_col.append((data[tkr][day].c - 
                             data[tkr][day].o - ff5['RF'][day])/
                             data[tkr][day].o)   
    dailyreturns[tkr] = dailyreturns_col

In [97]:
#calculate the days between now and the fama french data
today = datetime.date.today()
ff5end_date = ff5['Date'][0]
presidentsday = [datetime.date(2020, 2, 17)]
days_offset = wkdy.networkdays(ff5end_date.date(), today, presidentsday)

In [62]:
#data['STOCK'][index].letter
num_data = days_of_data-days_offset
X = pd.DataFrame(ff5[['Mkt-RF','SMB','HML','RMW','CMA']][0:num_data], columns=['Mkt-RF','SMB','HML','RMW','CMA'])
##NEXT UP - change it so you don't just regress it for one stock but iterate through for all stocks!
returns = pd.DataFrame(dailyreturns[days_offset:], columns=["Returns"])
y = returns["Returns"]

In [63]:
model = LinearRegression().fit(X, y)
betas = model.coef_

In [69]:

weights = np.ones(len(ff5))
i = 1
for j in range(len(ff5)) :
    weights[j] *= i
    i -= .00005
weights

array([1.     , 0.99995, 0.9999 , ..., 0.9384 , 0.93835, 0.9383 ])

In [78]:
factors = ['Mkt-RF','SMB','HML','RMW','CMA']
premiums = []
for fact in factors :
    av = np.average(ff5[fact])
    premiums.append(av)

In [3]:
#c of objective funct
#A_ub and b_ub
num_assets = 5
elem_vect = np.ones(num_assets)
#solution = opt.linprog(A_eq = elem_vect, b_eq = np.ones(1))

In [ ]:
clock = api.get_clock()
if clock.is_open :
    #trade away! 

In [11]:
data

{'AAPL': [Bar({   'c': 271.145,
      'h': 271.38,
      'l': 270.29,
      'o': 270.41,
      't': 1582923300,
      'v': 3548}),
  Bar({'c': 270.59, 'h': 271.5, 'l': 270.43, 'o': 271.07, 't': 1582923360, 'v': 6237}),
  Bar({'c': 270.93, 'h': 271, 'l': 270.38, 'o': 270.785, 't': 1582923420, 'v': 5854}),
  Bar({   'c': 272.22,
      'h': 272.375,
      'l': 271.08,
      'o': 271.08,
      't': 1582923480,
      'v': 8158}),
  Bar({   'c': 273.63,
      'h': 273.74,
      'l': 272.41,
      'o': 272.41,
      't': 1582923540,
      'v': 13792})],
 'NVDA': [Bar({'c': 266.38, 'h': 266.41, 'l': 265.73, 'o': 265.76, 't': 1582923300, 'v': 3042}),
  Bar({   'c': 266.275,
      'h': 266.59,
      'l': 265.97,
      'o': 266.27,
      't': 1582923360,
      'v': 2675}),
  Bar({'c': 266.84, 'h': 266.87, 'l': 266.21, 'o': 266.21, 't': 1582923420, 'v': 2652}),
  Bar({'c': 267.68, 'h': 267.68, 'l': 266.85, 'o': 266.85, 't': 1582923480, 'v': 2415}),
  Bar({   'c': 270.13,
      'h': 270.13,
      '

In [7]:
ff5

{0:             Mkt-RF   SMB   HML   RMW   CMA     RF
 Date                                             
 2015-03-09    0.37  0.09 -0.02  0.26 -0.03  0.000
 2015-03-10   -1.63  0.43 -0.47  0.09 -0.14  0.000
 2015-03-11   -0.04  0.62  0.52 -0.47 -0.03  0.000
 2015-03-12    1.28  0.40  0.50 -0.06  0.23  0.000
 2015-03-13   -0.57  0.19 -0.04 -0.10 -0.13  0.000
 ...            ...   ...   ...   ...   ...    ...
 2020-01-27   -1.56  0.27 -0.48 -0.59  0.05  0.006
 2020-01-28    1.02 -0.15 -0.42  0.17 -0.10  0.006
 2020-01-29   -0.10 -0.44 -0.95  0.25  0.19  0.006
 2020-01-30    0.34 -0.64  0.67 -0.39 -0.11  0.006
 2020-01-31   -1.74 -0.52 -0.34 -0.54 -0.69  0.006
 
 [1235 rows x 6 columns],
 'DESCR': 'F-F Research Data 5 Factors 2x3 daily\n-------------------------------------\n\nThis file was created by CMPT_ME_BEME_OP_INV_RETS_DAILY using the 202001 CRSP database. The 1-month TBill return is from Ibbotson and Associates, Inc.\n\n  0 : (1235 rows x 6 cols)'}